# Análisis de las preferencias de los viajeros y el impacto de los factores externos en los viajes compartidos en la ciudad de Chicago por *Carlos Horta* (carlosgim@gmail.com)

---

## Introducción

**Zuber** es una nueva empresa de viajes compartidos que recientemente se lanzó en Chicago.

El proyecto tiene como objetivo general identificar patrones en la información disponible en los diferentes conjuntos de datos.

Los objetivos específicos son los siguientes:

- Identificar las preferencias de los pasajeros.
- Determinar el impacto de los factores externos en los viajes.
- Probar la hipótesis de que "la duración promedio de los viajes desde el Loop hasta el Aeropuerto Internacional O'Hare varía los sábados lluviosos".

---

## Exploración de los datos

En esta primera parte del proyecto es necesario cargar las librerías a utilizar , así como los datasets, para luego realizar la preparación de los datos.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from scipy import stats as st
import plotly.express as px
import plotly.graph_objects as go

In [2]:
# Se cargan los dataset de dos formas distintas cada uno

try:
    trips = pd.read_csv("project_sql_result_01.csv")
except:
    trips = pd.read_csv("/datasets/project_sql_result_01.csv")
    
try:
    dropoff = pd.read_csv("project_sql_result_04.csv")
except:
    dropoff = pd.read_csv("/datasets/project_sql_result_04.csv")

try:
    loop = pd.read_csv("project_sql_result_07.csv")
except:
    loop = pd.read_csv("/datasets/project_sql_result_07.csv")


### Dataset trips 

In [3]:
# Revisemos el dataset llamado Trip el cual contiene el nombre de la empresa de taxis y el número de viajes de taxis de cada compañía el 15 y 16 de noviembre de 2017.

print(trips.info())
print('--------------------')
print(trips.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64 entries, 0 to 63
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   company_name  64 non-null     object
 1   trips_amount  64 non-null     int64 
dtypes: int64(1), object(1)
memory usage: 1.1+ KB
None
--------------------
       trips_amount
count     64.000000
mean    2145.484375
std     3812.310186
min        2.000000
25%       20.750000
50%      178.500000
75%     2106.500000
max    19558.000000


In [4]:
print(trips.head())
print('--------------------')
print(trips.isnull().sum())

                      company_name  trips_amount
0                        Flash Cab         19558
1        Taxi Affiliation Services         11422
2                 Medallion Leasin         10367
3                       Yellow Cab          9888
4  Taxi Affiliation Service Yellow          9299
--------------------
company_name    0
trips_amount    0
dtype: int64


Parece que los datos del dataset **trips** están correctos, porque tanto el nombre de la compañía como la cantidad de viajes aparecen de manera correcta, y son el tipo de dato esperable por cada columna. Así mismo, tampoco existen datos nulos en el dataset.

### Dataset dropoff

In [5]:
# El dataset dropoff contiene los barrios de Chicago donde finalizaron los viajes y el promedio de viajes que terminaron en cada barrio en noviembre de 2017.

print(dropoff.info())
print('--------------------')
print(dropoff.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   dropoff_location_name  94 non-null     object 
 1   average_trips          94 non-null     float64
dtypes: float64(1), object(1)
memory usage: 1.6+ KB
None
--------------------
       average_trips
count      94.000000
mean      599.953728
std      1714.591098
min         1.800000
25%        14.266667
50%        52.016667
75%       298.858333
max     10727.466667


In [6]:
print(dropoff.head())
print('--------------------')
print(dropoff.isnull().sum())

  dropoff_location_name  average_trips
0                  Loop   10727.466667
1           River North    9523.666667
2         Streeterville    6664.666667
3             West Loop    5163.666667
4                O'Hare    2546.900000
--------------------
dropoff_location_name    0
average_trips            0
dtype: int64


Como se observa, no existen datos nulos, pero es necesario modificar la columna del promedio de los viajes **average_trips** para convertirla de una en decimales en una de enteros.

In [7]:
dropoff['average_trips'] = np.floor(pd.to_numeric(dropoff['average_trips'], errors='coerce')).astype('Int64')
print(dropoff.info())
print('--------------------')
print(dropoff.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 94 entries, 0 to 93
Data columns (total 2 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   dropoff_location_name  94 non-null     object
 1   average_trips          94 non-null     Int64 
dtypes: Int64(1), object(1)
memory usage: 1.7+ KB
None
--------------------
  dropoff_location_name  average_trips
0                  Loop          10727
1           River North           9523
2         Streeterville           6664
3             West Loop           5163
4                O'Hare           2546


Como se observa, los datos sobre el promedio de los viajes (average_trips) están convertidos a formato de números enteros.

### Dataset Loop

In [8]:
# El dataset Loop contiene los viajes que iniciaron en el barrio Loop y terminaron en el Aeropuerto Internacional O'Hare.

print(loop.info())
print('--------------------')
print(loop.describe())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1068 entries, 0 to 1067
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   start_ts            1068 non-null   object 
 1   weather_conditions  1068 non-null   object 
 2   duration_seconds    1068 non-null   float64
dtypes: float64(1), object(2)
memory usage: 25.2+ KB
None
--------------------
       duration_seconds
count       1068.000000
mean        2071.731273
std          769.461125
min            0.000000
25%         1438.250000
50%         1980.000000
75%         2580.000000
max         7440.000000


In [9]:
print(loop.head())
print('--------------------')
print(loop.isnull().sum())

              start_ts weather_conditions  duration_seconds
0  2017-11-25 16:00:00               Good            2410.0
1  2017-11-25 14:00:00               Good            1920.0
2  2017-11-25 12:00:00               Good            1543.0
3  2017-11-04 10:00:00               Good            2512.0
4  2017-11-11 07:00:00               Good            1440.0
--------------------
start_ts              0
weather_conditions    0
duration_seconds      0
dtype: int64


Como se ve es necesario realizar dos acciones: 
* En **start_ts** es necesario extraer la fecha del viaje únicamente (ya que no interesa la hora de inicio del mismo) y
* En **weather_conditions** será necesario convertir cada "Good" a 0 y cada "Bad" a 1 para el análisis posterior de hipótesis,
* Queda pendiente revisar más adelante si la columna **duration_seconds** se deberá convertir a minutos para un análisis más sencillo.

In [10]:
# Iniciemos la extracción de la fecha de viaje de la columna start_ts

loop['start_ts'] =  pd.to_datetime(loop['start_ts'])
loop['date_finish'] = loop['start_ts'].dt.date
loop.head()

,start_ts,weather_conditions,duration_seconds,date_finish
0,2017-11-25 16:00:00,Good,2410.0,2017-11-25
1,2017-11-25 14:00:00,Good,1920.0,2017-11-25
2,2017-11-25 12:00:00,Good,1543.0,2017-11-25
3,2017-11-04 10:00:00,Good,2512.0,2017-11-04
4,2017-11-11 07:00:00,Good,1440.0,2017-11-11


In [11]:
# Continuemos con la conversión de "Good" a 0 y de "Bad" a 1 para su análisis posterior

loop['good_bad'] = loop['weather_conditions'].map({'Good':1, 'Bad':0})
loop.head()

,start_ts,weather_conditions,duration_seconds,date_finish,good_bad
0,2017-11-25 16:00:00,Good,2410.0,2017-11-25,1
1,2017-11-25 14:00:00,Good,1920.0,2017-11-25,1
2,2017-11-25 12:00:00,Good,1543.0,2017-11-25,1
3,2017-11-04 10:00:00,Good,2512.0,2017-11-04,1
4,2017-11-11 07:00:00,Good,1440.0,2017-11-11,1


## En resumen de las columnas

Se realizaron las siguientes observaciones y acciones en los conjuntos de datos:

- En el dataset **trips**:
  - Se identificó que todos los tipos de columna son correctos.
  - No se encontraron datos faltantes en el dataset.
  - Existe coherencia entre las columnas.

- En el dataset **dropoff**:
  - Se determinó que la columna **average_trips** tenía un tipo de columna incorrecto y se modificó a números enteros.
  - Se verificó que existe coherencia entre los datos en el dataset.

- En el dataset **Loop**:
  - Se creó una nueva columna con la fecha del viaje extraída de la columna existente.
  - Se creó otra columna donde se convirtió "Good" a 0 y "Bad" a 1, en caso de necesitarse en análisis posteriores.

---

---

## Análisis de los datos

Como primer paso es necesario identificar las 10 empresas con mayor número de viajes realizados, y también, identificar los 10 principales barrios en términos de finalización del recorrido.

In [12]:
trips_top_ten = trips.query('trips_amount > 5500')
fig = px.bar(trips_top_ten, y='trips_amount', x='company_name', text_auto='.2s', 
             title="Top 10 de compañías con la mayor cantidad de viajes realizados")
fig.show()

<div class="alert alert-block alert-warning">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

El argumento 'text_auto' no me dejó correr esta celda.
</div>

In [13]:
trips_top_ten = trips.query('trips_amount > 5500')
fig = px.bar(trips_top_ten, y='trips_amount', x='company_name',
             title="Top 10 de compañías con la mayor cantidad de viajes realizados")
fig.show()

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Muy buena gráfica de barras y buen uso de plotly.
</div>

Como se puede observar, la compañía **Flash Cab** fue la que más viajes realizó (19,558), seguida de **Taxi Affiliation Services** (11,422) y **Medallion **Leasin** (11,422).

Si se revisa de manera estadística esta información, se puede observar que existen 64 compañías de taxis, que la media es de 2,145 viajes, mientras que la mediana es de 178, con una desviación estándar de 3,812 viajes realizados, esta diferencia entre media y mediana da luz sobre la gran dispersión de los datos de este dataset.

In [14]:
dropoff_top_ten = dropoff.query('average_trips > 1250')
fig = px.bar(dropoff_top_ten, y='average_trips', x='dropoff_location_name', text_auto='.2s', 
             title="Top 10 de barrios por número de finalizaciones")
fig.show()

In [15]:
dropoff_top_ten = dropoff.query('average_trips > 1250')
fig = px.bar(dropoff_top_ten, y='average_trips', x='dropoff_location_name',
             title="Top 10 de barrios por número de finalizaciones")
fig.show()

Los cuatro barrios con más finalizaciones fueron Loop (10,727), River North (9,523), Streeterville (6,664) y West Loop (5,163), el resto de los barrios de llegada están por abajo de los 2,500 viajes, o sea, el cuarto barrio dobla en número al quinto (y siguientes) en el número de finalizaciones.

Estadísticamente, existen en el dataset 94 barrios con información, la media es de 94 viajes, la mediana de 52, y la desviación estándar de 1,714 viajes finalizados en su respectivo barrio.

## Pruebas de hipótesis

En este apartado se busca probar la siguiente hipótesis:
- *"La duración promedio de los viajes desde Loop hasta el Aeropuerto Internacional O'Hare cambia los sábados lluviosos"*.

In [16]:
# Es necesario tener dos dataset, uno donde las condiciones son buenas y otro para donde no lo son.

loop_good = loop.query('weather_conditions == "Good"')
loop_bad = loop.query('weather_conditions == "Bad"')


### Prueba de hipótesis #1

**Hipótesis nula**: *"La duración promedio de los viajes desde Loop hasta el Aeropuerto Internacional O'Hare no cambia en los sábados lluviosos"*

**Hipótesis alternativa**: *"La duración promedio de los viajes desde Loop hasta el Aeropuerto Internacional O'Hare cambia en los sábados lluviosos"*.

<div class="alert alert-block alert-success">
<b>Reviewer's comment</b> <a class="tocSkip"></a>

Bien planteaste correctamente las hipótesis de la prueba de hipótesis de igualdad de medias.
</div>

In [17]:
loop_good_seconds = loop_good['duration_seconds']
loop_bad_seconds = loop_bad['duration_seconds']

alpha = 0.05

result_test = st.ttest_ind(loop_good_seconds, loop_bad_seconds)
print('p-value:', result_test.pvalue)

if result_test.pvalue < alpha:
    print('Rechazamos la hipótesis nula')
else:
    print('No rechazamos la hipótesis nula')

p-value: 6.517970327099473e-12
Rechazamos la hipótesis nula


De acuerdo al test de verificación de hipótesis, se puede determinar que existe evidencia estadísticamente significativa sobre que la duración promedio de los viajes desde el barrio Loop hasta el Aeropuerto Internacional O'Hare cambia en los sábados lluviosos.

---

## Conclusión general

El proyecto titulado **"Análisis de las preferencias de los viajeros y el impacto de los factores externos en los viajes compartidos en la ciudad de Chicago"** se llevó a cabo con el objetivo de encontrar patrones en la información disponible, centrándose en dos objetivos específicos: 1) identificar las preferencias de los pasajeros y 2) identificar el impacto de los factores externos en los viajes.

El proyecto comenzó con un análisis exploratorio de datos utilizando SQL para determinar el número de viajes en taxi de cada empresa de viajes compartidos, así como el número de viajes finalizados en cada barrio de Chicago. Además, se seleccionaron los viajes que comenzaban en el barrio Loop y finalizaban en el Aeropuerto Internacional O'Hare, y se crearon columnas para categorizar las condiciones climáticas como "buenas" o "malas" y registrar la duración de los viajes en segundos.

A partir de los conjuntos de datos obtenidos mediante el trabajo en SQL, se realizaron los siguientes análisis y conclusiones:

- En el dataset **trips**, se determinó que todos los tipos de columna son correctos, no se encontraron datos faltantes y se observó coherencia entre las columnas.
- En el dataset **dropoff**, se identificó que la columna **average_trips** tenía un tipo de dato incorrecto, el cual se corrigió convirtiéndolo a números enteros. Además, se encontró coherencia en los datos de este dataset.
- En el dataset **Loop**, se crearon nuevas columnas para almacenar la fecha del viaje y para convertir las condiciones climáticas de "buenas" a 0 y de "malas" a 1, en caso de necesitarse en análisis posteriores.

Estas son las conclusiones específicas de cada uno de los tres conjuntos de datos:

- En el dataset **trips**, se encontró que la compañía **Flash Cab** realizó la mayor cantidad de viajes (19,558), seguida de **Taxi Affiliation Services** (11,422) y **Medallion Leasing** (11,422).
- En el dataset **dropoff**, se identificó que los barrios con mayor número de viajes finalizados fueron Loop (10,727), River North (9,523), Streeterville (6,664) y West Loop (5,163). El número de finalizaciones en el cuarto barrio duplica al quinto y los siguientes barrios presentan menos de 2,500 viajes finalizados.
- Mediante el análisis del dataset **Loop**, se pudo probar la hipótesis planteada y se encontró evidencia estadísticamente significativa de que la duración promedio de los viajes desde el barrio Loop hasta el Aeropuerto Internacional O'Hare varía en los sábados lluviosos.
